## Request and Inspect Weekly SPX-Options

First, load ib-ruby environment

In [1]:
require_relative '../lib/iruby_init.rb'


----> G    points to the Gateway-Instance
----> C    points to the Connection-Instance
----> Multible accounts detected
----> G.clients  points to the User-Accounts
Namespace is IB !


true

Check the Presence of predefined indices

In [2]:
Symbols::Index.all

[:a_d, :asx, :dax, :hsi, :minihsi, :spx, :stoxx, :tick, :trin, :vasx, :vdax, :vhsi, :vix, :volume, :vstoxx]

In [3]:
spx_market_price = IB::Symbols::Index.spx.market_price

TWS Error 354: Requested market data is not subscribed.Delayed market data is available.Error&CBOE/IND/Top&CBOE/IND/Top
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.


4227.26

_Note:_ If the request is fired before the market opening, the returned market price is the closing price of the previous day.

Next we define a range of strikes to consider as  4215 to 4340

In [4]:
spxw =  Option.new trading_class: :spxw, symbol: :spx, currency: :usd, expiry: 20210609, 
                   exchange: :smart, right: :put


spxw_options = spxw.verify{ |c| c if c.strike > spx_market_price - 15 && c.strike < spx_market_price + 15  }.compact
spxw_options.strike.sort

[4215.0, 4220.0, 4225.0, 4230.0, 4235.0, 4240.0]

Lets inspect these Options further.

The method _request_greeks_ requests Tickdata from the TWS-Server and collects the calculated greeks 


In [13]:
spxw_options.map{|c| c.request_greeks thread: true }.each &:join
puts "-"*50
puts spxw_options.sort_by(&:strike).map{|x| [x.to_human,x.greek.to_human].join( '-->')}.join("\n")

TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.


### Get ATM-Weeklies

There is a method `Contract#atm-options`. Its designed for monthly options

In [6]:
atm_options =  Symbols::Index.spx.atm_options

atm_options.keys    # Expiries of the Options

TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Error 200: No security definition has been found for the request


[2106, 2107, 2108, 2109, 2111, 2112, 2201, 2202, 2203, 2205, 2206, 2212, 2301]

In [7]:
atm_options[2106].to_human

["<Option: SPX 20210617 put 4225.0 SMART USD>"]

To get all available ATM-SPXW-Options we define a sample (as above) and specify the strike

In [15]:
spxw =  Option.new trading_class: :spxw, symbol: :spx, currency: :usd, strike: 4230, 
                   exchange: :smart, right: :put

spxw_chain =  spxw.verify.sort_by &:last_trading_day
puts spxw_chain.to_human


<Option: SPX 20210609 put 4230.0 SMART USD>
<Option: SPX 20210611 put 4230.0 SMART USD>
<Option: SPX 20210614 put 4230.0 SMART USD>
<Option: SPX 20210616 put 4230.0 SMART USD>
<Option: SPX 20210618 put 4230.0 SMART USD>
<Option: SPX 20210621 put 4230.0 SMART USD>
<Option: SPX 20210623 put 4230.0 SMART USD>
<Option: SPX 20210625 put 4230.0 SMART USD>
<Option: SPX 20210628 put 4230.0 SMART USD>
<Option: SPX 20210630 put 4230.0 SMART USD>
<Option: SPX 20210702 put 4230.0 SMART USD>
<Option: SPX 20210706 put 4230.0 SMART USD>
<Option: SPX 20210707 put 4230.0 SMART USD>
<Option: SPX 20210709 put 4230.0 SMART USD>
<Option: SPX 20210716 put 4230.0 SMART USD>
<Option: SPX 20210723 put 4230.0 SMART USD>
<Option: SPX 20210730 put 4230.0 SMART USD>
<Option: SPX 20210820 put 4230.0 SMART USD>
<Option: SPX 20210831 put 4230.0 SMART USD>
<Option: SPX 20210917 put 4230.0 SMART USD>
<Option: SPX 20210930 put 4230.0 SMART USD>
<Option: SPX 20211015 put 4230.0 SMART USD>
<Option: SPX 20211029 put 4230.0

In [16]:
spxw_chain.map{|o| o.request_greeks( thread: true)}.each &:join
puts spxw_chain.map{|o| "Expiry: #{o.expiry}   Market-price: #{o.greek.to_human}"}.join("\n")

TWS Warning 2119: Market data farm is connecting:usopt
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested mar